In [64]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['THE QUEEN _of_ HEARTS', '', '']


preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권합니다.

In [65]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()
    
    
    def count_character(data): 
        count = 0   
        for i in data :  
            count += len(i)
        return  count
    sentence_counter = sentence
    sentence_counter_data = sentence_counter.split() 
    #sentence_count = len(sentence_data)
 
    if count_character(sentence_counter_data) < 13:
        #print("공백을 제외한 문자수 : ",count_character(sentence_data)) 
        sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
        return sentence
    else :
        return False
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요. 



# <end>를 없애면 소스 문장, 첫 단어 <start>를 없애면 타겟 문장
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if preprocess_sentence(sentence) == False : continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:300]

False


['<start> simple simon <end>',
 '<start> toad and frog <end>',
 '<start> sat in a corner <end>',
 '<start> the wooing <end>',
 '<start> handy pandy <end>',
 '<start> blow wind blow <end>',
 '<start> blue bell boy <end>',
 '<start> buz and hum <end>',
 '<start> and so do we . <end>',
 '<start> else it was he . <end>',
 '<start> a and b and see <end>',
 '<start> doctor foster <end>',
 '<start> queen anne <end>',
 '<start> ho my kitten <end>',
 '<start> lavender blue <end>',
 '<start> cat and dog <end>',
 '<start> bobby shaft <end>',
 '<start> little maid <end>',
 '<start> bat , bat <end>',
 '<start> bat , bat , <end>',
 '<start> christmas <end>',
 '<start> peter white <end>',
 '<start> up pippen hill <end>',
 '<start> a falling out <end>',
 '<start> peg <end>',
 '<start> georgy porgy <end>',
 '<start> to market <end>',
 '<start> to buy a fat pig <end>',
 '<start> jiggety jig . <end>',
 '<start> to buy a fat hog <end>',
 '<start> jiggety jog . <end>',
 '<start> pancake day <end>',
 '<star

In [66]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    #tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    #for token in tensor:
    #if len(tensor)>15:continue
    #else:
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    
    print(tensor,tokenizer)
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print(len(tensor[0]))

[[   2 1257 1760 ...    0    0    0]
 [   2 1761   21 ...    0    0    0]
 [   2 1763   23 ...    0    0    0]
 ...
 [   2    3    0 ...    0    0    0]
 [   2    3    0 ...    0    0    0]
 [   2    3    0 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f26c47f4890>
13


In [67]:
print(tensor[:3, :10])

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

[[   2 1257 1760    3    0    0    0    0    0    0]
 [   2 1761   21 1762    3    0    0    0    0    0]
 [   2 1763   23   11 1015    3    0    0    0    0]]
1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : .
7 : you
8 : oh
9 : it
10 : me


In [68]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
# ## :, : ?의미를 확실히 알고가자

print(src_input[0])
print(tgt_input[0])

[   2 1257 1760    3    0    0    0    0    0    0    0    0]
[1257 1760    3    0    0    0    0    0    0    0    0    0]


In [69]:
from sklearn.model_selection import train_test_split


enc_train, enc_val, dec_train, dec_val = train_test_split(tgt_input, src_input, 
                                                                test_size=0.2, 
                                                                random_state = 1993, 
                                                                shuffle=True)


print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (11213, 12)
Target Train: (11213, 12)


In [74]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 12), (256, 12)), types: (tf.int32, tf.int32)>

In [88]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

model.summary()


optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.batch(256)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=15, validation_data=val_dataset)


Model: "text_generator_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_14 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_15 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_7 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
43/43 [==============================] - 5s 126ms/step - loss: 3.6475 - val_loss: 2.5632
Epoch 2/15
43/43 [==============================] - 5s 112ms/step - loss: 2.1925 - val_loss: 1.8122
Epoch 3/15
43/43 [==================

In [89]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [96]:
generate_text(model, tokenizer, init_sentence="<start>love you", max_len=20)

'<unk> you <start> you miss you miss you miss you miss you miss you miss you miss you miss you '

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
3. 텍스트 생성모델이 안정적으로 학습되었는가?
텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?


# 결론


데이터 15단어 이하로 맞추기 위해서 <start>, <end> 를 삭제하지 않고 문장 내에서 
단어를 카운팅 하여 줄이는게 핵심이었다. 
모델의 성능은 2.2이하로 만족할만한 수준이긴 하지만 작사 수준은 성능에 비해 많이 떨어지는듯하다
데이터 단어 수를 좀 더 높여서 학습한다면 만족할만한 작사수준이 나오지 않을까 예상해본다 .
